In [1]:
import pandas as pd
import numpy as np

# Importing the dataframe and getting its overview by printing first 10 rows.

In [2]:
df = pd.read_csv("retail_sales.csv")
df.head(10)

,Date,Store,Product,Category,Sales,Quantity
0,29-01-2023,Store_D,Product_3,Category_2,430.24,11
1,09-10-2023,Store_B,Product_1,Category_2,212.26,18
2,09-08-2023,Store_C,Product_1,Category_1,538.42,8
3,03-05-2023,Store_B,Product_2,Category_2,670.34,9
4,08-11-2023,Store_A,Product_3,Category_3,562.97,17
5,27-05-2023,Store_E,Product_3,Category_1,251.85,12
6,09-04-2023,Store_C,Product_4,Category_3,660.08,8
7,09-02-2023,Store_D,Product_4,Category_2,388.39,7
8,17-05-2023,Store_D,Product_2,Category_1,164.68,9
9,03-10-2023,Store_E,Product_5,Category_2,424.08,1


# Checking if the dataframe has any NaN values

In [3]:
df.loc[3:7, 'Quantity'] = np.nan
df

,Date,Store,Product,Category,Sales,Quantity
0,29-01-2023,Store_D,Product_3,Category_2,430.24,11.0
1,09-10-2023,Store_B,Product_1,Category_2,212.26,18.0
2,09-08-2023,Store_C,Product_1,Category_1,538.42,8.0
3,03-05-2023,Store_B,Product_2,Category_2,670.34,NaN
4,08-11-2023,Store_A,Product_3,Category_3,562.97,NaN
...,...,...,...,...,...,...
495,29-09-2023,Store_E,Product_1,Category_1,698.10,19.0
496,28-11-2023,Store_C,Product_1,Category_2,551.81,3.0
497,28-11-2023,Store_B,Product_5,Category_1,553.18,11.0
498,23-07-2023,Store_D,Product_5,Category_2,748.95,18.0


In [4]:
df.isna().sum()

Date        0
Store       0
Product     0
Category    0
Sales       0
Quantity    5
dtype: int64

In [5]:
df = df.fillna(df['Quantity'].mean())
df

,Date,Store,Product,Category,Sales,Quantity
0,29-01-2023,Store_D,Product_3,Category_2,430.24,11.000000
1,09-10-2023,Store_B,Product_1,Category_2,212.26,18.000000
2,09-08-2023,Store_C,Product_1,Category_1,538.42,8.000000
3,03-05-2023,Store_B,Product_2,Category_2,670.34,10.458586
4,08-11-2023,Store_A,Product_3,Category_3,562.97,10.458586
...,...,...,...,...,...,...
495,29-09-2023,Store_E,Product_1,Category_1,698.10,19.000000
496,28-11-2023,Store_C,Product_1,Category_2,551.81,3.000000
497,28-11-2023,Store_B,Product_5,Category_1,553.18,11.000000
498,23-07-2023,Store_D,Product_5,Category_2,748.95,18.000000


# Data Cleaning and Preparetion

#### Converting the date column to datetime format.

In [6]:
df['Date'] = pd.to_datetime(df["Date"], dayfirst = True)
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 500 entries, 0 to 499
Data columns (total 6 columns):
 #   Column    Non-Null Count  Dtype         
---  ------    --------------  -----         
 0   Date      500 non-null    datetime64[ns]
 1   Store     500 non-null    object        
 2   Product   500 non-null    object        
 3   Category  500 non-null    object        
 4   Sales     500 non-null    float64       
 5   Quantity  500 non-null    float64       
dtypes: datetime64[ns](1), float64(2), object(3)
memory usage: 23.6+ KB


Removing any duplicates if possible

In [7]:
df= df.drop_duplicates()
df.info() # There were no Duplicates as the number of columns have not decreased.

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 500 entries, 0 to 499
Data columns (total 6 columns):
 #   Column    Non-Null Count  Dtype         
---  ------    --------------  -----         
 0   Date      500 non-null    datetime64[ns]
 1   Store     500 non-null    object        
 2   Product   500 non-null    object        
 3   Category  500 non-null    object        
 4   Sales     500 non-null    float64       
 5   Quantity  500 non-null    float64       
dtypes: datetime64[ns](1), float64(2), object(3)
memory usage: 23.6+ KB


#### Handleing Outliars in the Sales column using the IQR method by replacing them with the median sales value.

In [8]:
# Calculating Quantiles and Bounds
Q1 = df["Sales"].quantile(0.25)
Q3 = df["Sales"].quantile(0.75)

IQR = Q3 - Q1

lower_bound = Q1 - (1.5 * IQR)
upper_bound = Q3 + (1.5 * IQR)
lower_bound, upper_bound 

(-473.19875, 1401.11125)

In [13]:
lower_criterian = (df["Sales"] > lower_bound)
upper_criterian = (df["Sales"] < upper_bound)
df = df[lower_criterian & upper_criterian]
df.info() # No rows were dropped.

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 500 entries, 0 to 499
Data columns (total 6 columns):
 #   Column    Non-Null Count  Dtype         
---  ------    --------------  -----         
 0   Date      500 non-null    datetime64[ns]
 1   Store     500 non-null    object        
 2   Product   500 non-null    object        
 3   Category  500 non-null    object        
 4   Sales     500 non-null    float64       
 5   Quantity  500 non-null    float64       
dtypes: datetime64[ns](1), float64(2), object(3)
memory usage: 23.6+ KB


# Sales Analysis

#### Calculate total sale of each store and display top 5 stores by sales

In [26]:
# Finding unique values in stores column
stores = df["Store"].unique()
sales = []
for store in stores:
    sales.append(df[df["Store"] == store]["Sales"].sum())
temp_li = np.array([[i,j] for i, j in zip(stores, sales)])
pd.DataFrame(temp_li, columns = ["Store", "Sales"]).sort_values(by = ["Sales"]).head()



,Store,Sales
2,Store_C,44523.58
4,Store_E,45513.99
0,Store_D,45760.31999999999
1,Store_B,50726.71000000001
3,Store_A,52090.36


#### Which product Category had highest average sales

In [49]:
temp = df.groupby('Category')['Sales'].mean().reset_index()
temp = temp.sort_values(by = 'Sales', ascending = False).reset_index()
temp.loc[0, ["Category","Sales"]]


KeyError: 'Category'

#### The best product in terms of quantity sold

In [48]:
df = df.groupby("Product")["Quantity"].sum().reset_index()
df.sort_values(by = 'Quantity', ascending = False)

,Product,Quantity
0,Product_1,1222.000000
3,Product_4,1023.917172
2,Product_3,1008.917172
1,Product_2,992.458586
4,Product_5,982.000000


# Ranking Analysis